# Vignere Cipher Tool

This notebook contains the tools to encode and decode text with the Vignere Cipher, as well as breaking the cipher by decrypting ciphertext using frequency analysis - enjoy!

The below class is our encoder/decoder - this requires uppercase text, and can work for other alphabet if included in the input

In [42]:
import re
import numpy as np

class VigenereCipher(object):
    def __init__(self, key, alphabet):
        self.key = key
        self.letters = re.findall('.',alphabet)
        nos = np.linspace(0,len(alphabet),len(alphabet)+1).astype(int)
        self.decoded = {alphabet[i]: nos[i] for i in range(len(alphabet))}
        pass
    
    def encode(self,text):
        if text.islower() == True:
            return(text)
        list_ = np.linspace(1,len(self.letters),len(self.letters))
        message = []
        key = self.key*(int(len(text)/len(self.key))+1)
        for j in range(len(text)):
            key_letter = key[j]
            shift = self.decoded[key_letter]
            try:
                letter = self.letters[(self.letters.index(text[j])+shift) %len(self.letters)]
            except:
                letter = text[j]
            message.append(letter)
        return(''.join(message))

    def decode(self, text):
        if text.islower() == True:
            return(text)
        list_ = np.linspace(1,len(self.letters),len(self.letters))
        message = []
        key = self.key*(int(len(text)/len(self.key))+1)
        for j in range(len(text)):
            key_letter = key[j]
            shift = -self.decoded[key_letter]
            try:
                letter = self.letters[(self.letters.index(text[j])+shift) %len(self.letters)]
            except:
                letter = text[j]
            message.append(letter)
        return(''.join(message))

This here is the function to get key length - this function uses the index of coincidence  - see https://en.wikipedia.org/wiki/Index_of_coincidence#Example 

In summary - try different key sizes, and if the key size happens to have been the same as the assumed number of columns, then all the letters within a single column will have been enciphered using the same key letter and therefore the corresponding set of ciphertext letters should have a roughness of frequency distribution similar to that of English, although the letter identities have been shifted.

Therefore, if we compute the aggregate delta I.C. for all columns, it should be around 1.73. On the other hand, if we have incorrectly guessed the key size (number of columns), the aggregate delta I.C. should be around 1.00

In [45]:
import numpy as np
import re
from itertools import count

def get_key_length(cipher_text,max_key_length):
    cipher_text = cipher_text.upper()
    cipher_text = re.sub('[^A-Z]+', '', cipher_text)
    alphabet = np.unique(re.findall('.',cipher_text))
    def get_ic(cipher_text,alphabet,n):
        cipher_text = (re.findall('.'*n,cipher_text))
        
        def get_sub(x):
            new_text = []
            freq = []
            for i in range(len(cipher_text)):
                new_text.append(cipher_text[i][x])
            [freq.append(new_text.count(letter)) for letter in alphabet]
            #count frequencies of letters in text
            num = 0
            dem = len(new_text)*((len(new_text)-1)/26)
            for i in freq:
                num = num + (i*(i-1))
            return(num/dem)
        ic = 0
        sub_ic = 0
        #calculate I.C.
        for i in range(n):
            sub_ic = sub_ic + get_sub(i)
        ic = sub_ic/n
        #how close to english language I.C. of 1.73
        return(np.abs(ic-1.73))
    vals = []
    key_lengths = []
    for i in range(max_key_length):
        i = i+1
        #Assume key length not shorter than 3
        if i <3:
            continue
        vals.append(get_ic(cipher_text,alphabet,i))
        key_lengths.append(i)
    return(key_lengths[np.array(vals).argmin()])
    

This below function is the cipher breaker, which relies on the previous function to first obtain the key length. This code takes the frequencies of letters in the english language and tries to match it up with the shifted alphabet, and the shift which matches for each nth letter in the key gives that letter of the key

In [47]:
import matplotlib.pyplot as plt
import re
from collections import OrderedDict
from itertools import islice, cycle
import numpy as np

def shift_dict(dct, shift):
    shift %= len(dct)
    return OrderedDict(
        (k, v)
        for k, v in zip(dct.keys(), islice(cycle(dct.values()), shift, None))
    )

def get_key(text,len_key):

    split = (re.findall('.'*len_key,text))
    alphabet = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
    decode = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,
                'I':8,'J':9,'K':10,'L':11,'M':12,'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,
                'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,'Z':25}
    english = {'A':0.082,'B':0.015,'C':0.028,'D':0.043,'E':0.13,'F':0.022,'G':0.02,'H':0.061,
                'I':0.07,'J':0.0015,'K':0.0077,'L':0.04,'M':0.024,'N':0.067,'O':0.075,'P':0.019,'Q':0.0095,'R':0.06,'S':0.063,
                'T':0.091,'U':0.028,'V':0.0098,'W':0.024,'X':0.0015,'Y':0.02,'Z':0.00074}

    frequencies ={'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,
                'I':0,'J':0,'K':0,'L':0,'M':0,'N':0,'O':0,'P':0,'Q':0,'R':0,'S':0,
                'T':0,'U':0,'V':0,'W':0,'X':0,'Y':0,'Z':0}


    def frequency_analysis(split,column):
        first_list = []

        for i in split:
            first_list.append(i[column])

        for letter in (alphabet):
            frequencies[letter] = first_list.count(letter)/len(first_list)
        sums =np.array([])
        for i in range(26):
            shifted = shift_dict(english,i)
            sums = np.append(sums,sum(abs(np.array(list(frequencies.values()))-np.array(list(shifted.values())))))
        min_ = (np.argmin(sums))
        new = (26-min_) %26

        #option to plot the shift figures here, uncomment if liked
        '''
        plt.figure()
        plt.bar(*zip(*frequencies.items()),label = 'text')
        plt.bar(*zip(*english.items()),label='english',alpha=0.5)
        shifted = shift_dict(english,min_)
        plt.bar(*zip(*shifted.items()),label='shifted',alpha=0.5)
        plt.legend()
        '''
    
        max_let = max(frequencies, key=frequencies.get)
        key_letter_1 = [let for let, val in decode.items() if val == new]
        return(key_letter_1)

    key = []
    for i in range(len_key):
        key_letter = frequency_analysis(split,i)
        key.append(key_letter[0])

    return(''.join(key))


In [48]:
def break_code(cipher_text):
    key_length = get_key_length(cipher_text,10) #assuming a max key length of 10, but this can be increased
    key = get_key(cipher_text,key_length)
    return(key)

Finally, we see an example here of some ciphertext encoded with a key of unknown length. The key is obtained using the I.C. and frequency analysis functions, then the ciphertext is decoded using our encoder/decoder class

In [50]:
cipher_text ='NSWXARWJGEXIJCZWUZLOAWFJFTUIMUVFEWHWPEEVVCYENYSGVVECSRZLGFDRZBPKWPMIYTFFGQDRJOKOTWWIWNVKUOBEXOLLZFDCOWZLJCXXQSZFITUIMUVFVLVEJDKZGSVWWYNANZKERERFKRLSOYEUTOWMYLVLVSUJNEHMGBFCEFKZGSVWWYZKCPRYPTYWHFHUQEELWGHSBXISAGWSPRVSVNHFNAJAPEDXWRUAHTHVANKSWHKSNSYSXSKEXIKKYVLGDCRFDSUIBLVUVSGMJTYWKFXWAOWDGHWINSYWOWQKSAPKYFLXENXKVMOIBOIWZOPTHEZKXWVMXLPVKTIINEELHFRQBALDMBHVOLVLVSUFEGISOHUMCRREYCUHBRVIWSQGEEJOQFGPANXLJOQHOEELGBFIHEEYVVFEJBVUCZFYHAKWFTRVOPVUKTLGWUKZQFVEJDLKGRWSLRFNGCUHESGJQJHEQTYGTGKMLOWLGLWWAVVFHCUEQTYGTGZLKSVKVMOIOAIWPCWWKDZNGFJIJTRUEIUEPERNGFDKALVLVSUJNEHMGBFMASTSPCQPUBVYNSDRADSQCBDPUZZFIOOENGVSOCXRPOWJGDUIOEELCHLZATVPVKLXDTYWCJDMHASANWWCKFDGFSURYODHWHLRCAEVECOPACKAQBVSBLRJISWITTTGTDRVWSLUJQDPYUCSVWRROAIWGOVMHYDSFSHBWMGDGGFEJBVVTOZRBRFECJDVEEKQQTVSQRTWUDUIOSIWRCUXENXJGZLKEOLKVSAXOSTAGBWMBITLGLWWWNUYGBHVWLWAEHLSJAEVVVHVAAIWFWIJARVFESVIOPVUKOOPUFFJISQINACXKQWMKNNAVVWLAPFKKHLSJOWZCBGMSIKZJPHGKMZFIARVACFEOCQLARSWTHVDEMZFJWVGHAJKKQLRPRFVWQWSNYTJADWSCRRHJMWITTTGFSVEJDJWEFHXSRZLKBJKEVVKVVHIJGCAUVOIPTVJHFHUQEEUAGHUQEEUGOVIPAFFTWVLZLWUOIJCLWSNMXAUVTYWOCVXYODEQBOIPTVJROLVOAJLJVHEJRVWTWQSJAKFFGWIOEEGHHHIZOILKVLEOTFSPRWLAMFKVQRQIOEVQIEPADCWVHHVOAJDNSHWOOFLVTIVNNEHRQFXDEKGRHZIHVVDGHWINSTGODUMOERTQIWSBTYWVCWEHUJSISWLATFHGWJLPLVLVSUWYODHTWVIWBFMVCIXDEKGVOOYOAXWNSWXARWJGEXIJCPSUOIYJCKAQBRJNAECEOQFAFZLVSGAALCTAGHZARRDTOQOBUEUVWRROWZLJHKIPWFHCFDQATVJECFLKBVLCFDRGFLFEHLSJBVAPUWLABVKVOQSPHVJTOQOBUEUVWRRSIKZPCDHFUJLCPOIBRVWROUEIEKWTOOWKFZLUHKIHEKLGFIVAQLWPQBHESKJKPXXEOEJGOVSJASDAKHPHTYWUOPIBUEUVWRRDAJTGSQYOEULQTLXPHVSOWQSWCZVHFHUQEEUAWQTNOKWKBVIMUVFESVEOPPMUWQKPHVNKQFMLHVJQFVSIEFLJSUGEPYWTPDWADFFCGWVWDUDKBJGDETCGFESWRULADLGWLCQWGHWWMEWOCQMYSLUJOVEOIELQSUVZRFHRWQKPHVKGQRRZRKGTSPIIBVJVVHXKPVAIVWGDAISEHHVOTYWWGHSBLVLVSUJNEHMGBFMASRFFTUIMUVFEMDRWLPKKGSPWYJSHIQHWMVFVOOVKLVAPQUCLTFYTOPWWNUKGJHVWLNGTRSYVZCWIOPIOIEUNIGMJGYSPUPEJSTJCPEPAAEVVVHXALVNKGLSJGREGGKSSWYWGZRJBOILWBHSJEFXVVHIWRCAGGWHASTJKDWMKNZFEZDWOITSNZLXARRLWFHSBAGHNMLRCTYWMBRAHEUYGCIIJGCAUVOIPTVJHFHUQEEUAHRWKLMAPUDGNYGLQUUEIIJXQIQHENVSRCHWBADGWGVXKRPLJSJSHDSMIKKINEKZGAHXDOUAUGXGYEJKHIOPUAGHNWHHPOUWEWSLARREGGVECEZFUHUYYTZFICQXDENZGFHEXOLLUCIEPRVSUIUIDIUVGBECYAGLCWQOEDUDGHWINFIWSIHRYIVKJOGEOTIGPUHJBETLQBWLADVKKUQSBSFEGYHCXORJFZDCKUKKVVHQKSKXTSTYANKDGHWINSRJGCQXDESGVHRQNONGHHKIXLZUMSQWZEIXGFWCLENJKHHVWNULJSKSIEIGYCIXDEUNQFDOOIDHNWIMADBWAPREND'
key = break_code(cipher_text)
c = VigenereCipher(key, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ')
print(c.decode(cipher_text))

LETTERFREQUENCIESLIKEWORDFREQUENCIESTENDTOVARYBOTHBYWRITERANDBYSUBJECTONECANNOTWRITEANESSAYABOUTXRAYSWITHOUTUSINGFREQUENTXSANDTHEESSAYWILLHAVEANIDIOSYNCRATICLETTERFREQUENCYIFTHEESSAYISABOUTTHEFREQUENTUSEOFXRAYSTOTREATZEBRASINQATARDIFFERENTAUTHORSHAVEHABITSWHICHCANBEREFLECTEDINTHEIRUSEOFLETTERSHEMINGWAYSWRITINGSTYLEFOREXAMPLEISVISIBLYDIFFERENTFROMFAULKNERSLETTERBIGRAMTRIGRAMWORDFREQUENCIESWORDLENGTHANDSENTENCELENGTHCANBECALCULATEDFORSPECIFICAUTHORSANDUSEDTOPROVEORDISPROVEAUTHORSHIPOFTEXTSEVENFORAUTHORSWHOSESTYLESARENOTSODIVERGENTACCURATEAVERAGELETTERFREQUENCIESCANONLYBEGLEANEDBYANALYZINGALARGEAMOUNTOFREPRESENTATIVETEXTWITHTHEAVAILABILITYOFMODERNCOMPUTINGANDCOLLECTIONSOFLARGETEXTCORPORASUCHCALCULATIONSAREEASILYMADEEXAMPLESCANBEDRAWNFROMAVARIETYOFSOURCESPRESSREPORTINGRELIGIOUSTEXTSSCIENTIFICTEXTSANDGENERALFICTIONANDTHEREAREDIFFERENCESESPECIALLYFORGENERALFICTIONWITHTHEPOSITIONOFHANDIWITHHBECOMINGMORECOMMONHERBERTSZIMINHISCLASSICINTRODUCTORYCRYPTOGRAPHYTEXTCODESANDSECRETWRITINGGIVESTHEENGLI